In [1]:
import colorsys
import os
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
import glob
from keras.utils import multi_gpu_model
import pandas as pd

Using TensorFlow backend.
c:\users\user\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\anaconda3\envs\tensor

In [2]:
class YOLO(object):
    _defaults = {
        "model_path": 'model_data/logs/trained_weights_final.h5',
        "anchors_path": 'model_data/yolo_anchors.txt',
        "classes_path": 'model_data/my_classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_image(self, image, jpgfile):
        start = timer()
        jpgfile = jpgfile.replace('yolo3\\VOCdevkit\\VOC2007\\class\\', '')
        
        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.

        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300
        
        list_num =[]
        
        if len(out_boxes)==0:
            return
        else:
            for i, c in reversed(list(enumerate(out_classes))):
                predicted_class = self.class_names[c]
                box = out_boxes[i]
                score = out_scores[i]

                label = '{} {:.2f}'.format(predicted_class, score)
                draw = ImageDraw.Draw(image)
                label_size = draw.textsize(label, font)

                top, left, bottom, right = box
                top = max(0, np.floor(top + 0.5).astype('int32'))
                left = max(0, np.floor(left + 0.5).astype('int32'))
                bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
                right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
                
                list_num1 =[jpgfile, predicted_class, left, top, right, bottom, score]
                list_num.append(list_num1)
                
            


            #if top - label_size[1] >= 0:
                #text_origin = np.array([left, top - label_size[1]])
            #else:
                #text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            #for i in range(thickness):
                #draw.rectangle(
                    #[left + i, top + i, right - i, bottom - i],
                    #outline=self.colors[c])
            #draw.rectangle(
                #[tuple(text_origin), tuple(text_origin + label_size)],
                #fill=self.colors[c])
            #draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            #del draw
        return list_num
        #end = timer()
        #print(end - start)
        
    def close_session(self):
        self.sess.close()
    

def detect_video(yolo, video_path, output_path=""):
    import cv2
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
    video_fps       = vid.get(cv2.CAP_PROP_FPS)
    video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    isOutput = True if output_path != "" else False
    if isOutput:
        print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
        out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    while True:
        return_value, frame = vid.read()
        image = Image.fromarray(frame)
        image = yolo.detect_image(image)
        result = np.asarray(image)
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
        cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
        cv2.namedWindow("result", cv2.WINDOW_NORMAL)
        cv2.imshow("result", result)
        if isOutput:
            out.write(result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    yolo.close_session()
        
    

In [3]:
list_all=[]
i=1
yolo=YOLO()
path = "yolo3\VOCdevkit\VOC2007\class\*.jpg"
sum_=len(glob.glob(path))
for jpgfile in glob.glob(path):
    img = Image.open(jpgfile)
    list_num = yolo.detect_image(img,jpgfile)
    list_all.append(list_num)
    print(str(i)+'/'+str(sum_))
    i=i+1
yolo.close_session()








model_data/logs/trained_weights_final.h5 model, anchors, and classes loaded.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
(416, 416, 3)
Found 2 boxes for img
1/2248
(416, 416, 3)
Found 0 boxes for img
2/2248
(416, 416, 3)
Found 1 boxes for img
3/2248
(416, 416, 3)
Found 1 boxes for img
4/2248
(416, 416, 3)
Found 2 boxes for img
5/2248
(416, 416, 3)
Found 0 boxes for img
6/2248
(416, 416, 3)
Found 0 boxes for img
7/2248
(416, 416, 3)
Found 1 boxes for img
8/2248
(416, 416, 3)
Found 1 boxes for img
9/2248
(416, 416, 3)
Found 1 boxes for img
10/2248
(416, 416, 3)
Found 0 boxes for img
11/2248
(416, 416, 3)
Found 4 boxes for img
12/2248
(416, 416, 3)
Found 0 boxes for img
13/2248
(416, 416, 3)
Found 5 boxes for img
14/2248
(416, 416, 3)
Found 0 boxes for img
15/2248
(416, 416, 3)
Found 0 boxes for img
16/2248
(416, 416, 3)
Found 0 boxes for img
17/2248
(416, 416, 3)
Found 1 boxes for img
18/2248
(416, 416, 3)
Found 1 boxes for img
19/

Found 0 boxes for img
142/2248
(416, 416, 3)
Found 1 boxes for img
143/2248
(416, 416, 3)
Found 3 boxes for img
144/2248
(416, 416, 3)
Found 0 boxes for img
145/2248
(416, 416, 3)
Found 3 boxes for img
146/2248
(416, 416, 3)
Found 0 boxes for img
147/2248
(416, 416, 3)
Found 5 boxes for img
148/2248
(416, 416, 3)
Found 0 boxes for img
149/2248
(416, 416, 3)
Found 1 boxes for img
150/2248
(416, 416, 3)
Found 0 boxes for img
151/2248
(416, 416, 3)
Found 0 boxes for img
152/2248
(416, 416, 3)
Found 6 boxes for img
153/2248
(416, 416, 3)
Found 1 boxes for img
154/2248
(416, 416, 3)
Found 0 boxes for img
155/2248
(416, 416, 3)
Found 2 boxes for img
156/2248
(416, 416, 3)
Found 1 boxes for img
157/2248
(416, 416, 3)
Found 3 boxes for img
158/2248
(416, 416, 3)
Found 0 boxes for img
159/2248
(416, 416, 3)
Found 1 boxes for img
160/2248
(416, 416, 3)
Found 2 boxes for img
161/2248
(416, 416, 3)
Found 5 boxes for img
162/2248
(416, 416, 3)
Found 10 boxes for img
163/2248
(416, 416, 3)
Found 0 b

Found 0 boxes for img
325/2248
(416, 416, 3)
Found 2 boxes for img
326/2248
(416, 416, 3)
Found 0 boxes for img
327/2248
(416, 416, 3)
Found 0 boxes for img
328/2248
(416, 416, 3)
Found 0 boxes for img
329/2248
(416, 416, 3)
Found 1 boxes for img
330/2248
(416, 416, 3)
Found 0 boxes for img
331/2248
(416, 416, 3)
Found 1 boxes for img
332/2248
(416, 416, 3)
Found 6 boxes for img
333/2248
(416, 416, 3)
Found 1 boxes for img
334/2248
(416, 416, 3)
Found 0 boxes for img
335/2248
(416, 416, 3)
Found 0 boxes for img
336/2248
(416, 416, 3)
Found 1 boxes for img
337/2248
(416, 416, 3)
Found 4 boxes for img
338/2248
(416, 416, 3)
Found 0 boxes for img
339/2248
(416, 416, 3)
Found 1 boxes for img
340/2248
(416, 416, 3)
Found 1 boxes for img
341/2248
(416, 416, 3)
Found 0 boxes for img
342/2248
(416, 416, 3)
Found 0 boxes for img
343/2248
(416, 416, 3)
Found 1 boxes for img
344/2248
(416, 416, 3)
Found 1 boxes for img
345/2248
(416, 416, 3)
Found 0 boxes for img
346/2248
(416, 416, 3)
Found 5 bo

Found 0 boxes for img
508/2248
(416, 416, 3)
Found 0 boxes for img
509/2248
(416, 416, 3)
Found 0 boxes for img
510/2248
(416, 416, 3)
Found 0 boxes for img
511/2248
(416, 416, 3)
Found 2 boxes for img
512/2248
(416, 416, 3)
Found 0 boxes for img
513/2248
(416, 416, 3)
Found 0 boxes for img
514/2248
(416, 416, 3)
Found 1 boxes for img
515/2248
(416, 416, 3)
Found 2 boxes for img
516/2248
(416, 416, 3)
Found 2 boxes for img
517/2248
(416, 416, 3)
Found 2 boxes for img
518/2248
(416, 416, 3)
Found 1 boxes for img
519/2248
(416, 416, 3)
Found 1 boxes for img
520/2248
(416, 416, 3)
Found 1 boxes for img
521/2248
(416, 416, 3)
Found 0 boxes for img
522/2248
(416, 416, 3)
Found 1 boxes for img
523/2248
(416, 416, 3)
Found 0 boxes for img
524/2248
(416, 416, 3)
Found 5 boxes for img
525/2248
(416, 416, 3)
Found 0 boxes for img
526/2248
(416, 416, 3)
Found 0 boxes for img
527/2248
(416, 416, 3)
Found 0 boxes for img
528/2248
(416, 416, 3)
Found 0 boxes for img
529/2248
(416, 416, 3)
Found 1 bo

Found 0 boxes for img
690/2248
(416, 416, 3)
Found 0 boxes for img
691/2248
(416, 416, 3)
Found 2 boxes for img
692/2248
(416, 416, 3)
Found 1 boxes for img
693/2248
(416, 416, 3)
Found 0 boxes for img
694/2248
(416, 416, 3)
Found 3 boxes for img
695/2248
(416, 416, 3)
Found 0 boxes for img
696/2248
(416, 416, 3)
Found 0 boxes for img
697/2248
(416, 416, 3)
Found 1 boxes for img
698/2248
(416, 416, 3)
Found 0 boxes for img
699/2248
(416, 416, 3)
Found 0 boxes for img
700/2248
(416, 416, 3)
Found 0 boxes for img
701/2248
(416, 416, 3)
Found 0 boxes for img
702/2248
(416, 416, 3)
Found 0 boxes for img
703/2248
(416, 416, 3)
Found 0 boxes for img
704/2248
(416, 416, 3)
Found 1 boxes for img
705/2248
(416, 416, 3)
Found 0 boxes for img
706/2248
(416, 416, 3)
Found 0 boxes for img
707/2248
(416, 416, 3)
Found 0 boxes for img
708/2248
(416, 416, 3)
Found 0 boxes for img
709/2248
(416, 416, 3)
Found 0 boxes for img
710/2248
(416, 416, 3)
Found 0 boxes for img
711/2248
(416, 416, 3)
Found 2 bo

...

Found 0 boxes for img
1412/2248
(416, 416, 3)
Found 0 boxes for img
1413/2248
(416, 416, 3)
Found 0 boxes for img
1414/2248
(416, 416, 3)
Found 3 boxes for img
1415/2248
(416, 416, 3)
Found 0 boxes for img
1416/2248
(416, 416, 3)
Found 0 boxes for img
1417/2248
(416, 416, 3)
Found 1 boxes for img
1418/2248
(416, 416, 3)
Found 0 boxes for img
1419/2248
(416, 416, 3)
Found 0 boxes for img
1420/2248
(416, 416, 3)
Found 0 boxes for img
1421/2248
(416, 416, 3)
Found 1 boxes for img
1422/2248
(416, 416, 3)
Found 0 boxes for img
1423/2248
(416, 416, 3)
Found 1 boxes for img
1424/2248
(416, 416, 3)
Found 3 boxes for img
1425/2248
(416, 416, 3)
Found 5 boxes for img
1426/2248
(416, 416, 3)
Found 0 boxes for img
1427/2248
(416, 416, 3)
Found 7 boxes for img
1428/2248
(416, 416, 3)
Found 4 boxes for img
1429/2248
(416, 416, 3)
Found 0 boxes for img
1430/2248
(416, 416, 3)
Found 0 boxes for img
1431/2248
(416, 416, 3)
Found 0 boxes for img
1432/2248
(416, 416, 3)
Found 0 boxes for img
1433/2248
(4

Found 1 boxes for img
1591/2248
(416, 416, 3)
Found 1 boxes for img
1592/2248
(416, 416, 3)
Found 0 boxes for img
1593/2248
(416, 416, 3)
Found 0 boxes for img
1594/2248
(416, 416, 3)
Found 0 boxes for img
1595/2248
(416, 416, 3)
Found 0 boxes for img
1596/2248
(416, 416, 3)
Found 0 boxes for img
1597/2248
(416, 416, 3)
Found 0 boxes for img
1598/2248
(416, 416, 3)
Found 1 boxes for img
1599/2248
(416, 416, 3)
Found 4 boxes for img
1600/2248
(416, 416, 3)
Found 0 boxes for img
1601/2248
(416, 416, 3)
Found 1 boxes for img
1602/2248
(416, 416, 3)
Found 3 boxes for img
1603/2248
(416, 416, 3)
Found 0 boxes for img
1604/2248
(416, 416, 3)
Found 2 boxes for img
1605/2248
(416, 416, 3)
Found 2 boxes for img
1606/2248
(416, 416, 3)
Found 2 boxes for img
1607/2248
(416, 416, 3)
Found 1 boxes for img
1608/2248
(416, 416, 3)
Found 1 boxes for img
1609/2248
(416, 416, 3)
Found 0 boxes for img
1610/2248
(416, 416, 3)
Found 2 boxes for img
1611/2248
(416, 416, 3)
Found 0 boxes for img
1612/2248
(4

Found 1 boxes for img
1770/2248
(416, 416, 3)
Found 0 boxes for img
1771/2248
(416, 416, 3)
Found 1 boxes for img
1772/2248
(416, 416, 3)
Found 2 boxes for img
1773/2248
(416, 416, 3)
Found 1 boxes for img
1774/2248
(416, 416, 3)
Found 2 boxes for img
1775/2248
(416, 416, 3)
Found 0 boxes for img
1776/2248
(416, 416, 3)
Found 2 boxes for img
1777/2248
(416, 416, 3)
Found 2 boxes for img
1778/2248
(416, 416, 3)
Found 4 boxes for img
1779/2248
(416, 416, 3)
Found 1 boxes for img
1780/2248
(416, 416, 3)
Found 1 boxes for img
1781/2248
(416, 416, 3)
Found 1 boxes for img
1782/2248
(416, 416, 3)
Found 3 boxes for img
1783/2248
(416, 416, 3)
Found 1 boxes for img
1784/2248
(416, 416, 3)
Found 2 boxes for img
1785/2248
(416, 416, 3)
Found 3 boxes for img
1786/2248
(416, 416, 3)
Found 3 boxes for img
1787/2248
(416, 416, 3)
Found 1 boxes for img
1788/2248
(416, 416, 3)
Found 1 boxes for img
1789/2248
(416, 416, 3)
Found 1 boxes for img
1790/2248
(416, 416, 3)
Found 1 boxes for img
1791/2248
(4

Found 1 boxes for img
1949/2248
(416, 416, 3)
Found 1 boxes for img
1950/2248
(416, 416, 3)
Found 2 boxes for img
1951/2248
(416, 416, 3)
Found 2 boxes for img
1952/2248
(416, 416, 3)
Found 3 boxes for img
1953/2248
(416, 416, 3)
Found 1 boxes for img
1954/2248
(416, 416, 3)
Found 0 boxes for img
1955/2248
(416, 416, 3)
Found 1 boxes for img
1956/2248
(416, 416, 3)
Found 1 boxes for img
1957/2248
(416, 416, 3)
Found 1 boxes for img
1958/2248
(416, 416, 3)
Found 4 boxes for img
1959/2248
(416, 416, 3)
Found 1 boxes for img
1960/2248
(416, 416, 3)
Found 0 boxes for img
1961/2248
(416, 416, 3)
Found 0 boxes for img
1962/2248
(416, 416, 3)
Found 4 boxes for img
1963/2248
(416, 416, 3)
Found 2 boxes for img
1964/2248
(416, 416, 3)
Found 1 boxes for img
1965/2248
(416, 416, 3)
Found 2 boxes for img
1966/2248
(416, 416, 3)
Found 1 boxes for img
1967/2248
(416, 416, 3)
Found 3 boxes for img
1968/2248
(416, 416, 3)
Found 2 boxes for img
1969/2248
(416, 416, 3)
Found 0 boxes for img
1970/2248
(4

Found 0 boxes for img
2128/2248
(416, 416, 3)
Found 3 boxes for img
2129/2248
(416, 416, 3)
Found 3 boxes for img
2130/2248
(416, 416, 3)
Found 3 boxes for img
2131/2248
(416, 416, 3)
Found 3 boxes for img
2132/2248
(416, 416, 3)
Found 0 boxes for img
2133/2248
(416, 416, 3)
Found 0 boxes for img
2134/2248
(416, 416, 3)
Found 4 boxes for img
2135/2248
(416, 416, 3)
Found 0 boxes for img
2136/2248
(416, 416, 3)
Found 1 boxes for img
2137/2248
(416, 416, 3)
Found 1 boxes for img
2138/2248
(416, 416, 3)
Found 2 boxes for img
2139/2248
(416, 416, 3)
Found 2 boxes for img
2140/2248
(416, 416, 3)
Found 3 boxes for img
2141/2248
(416, 416, 3)
Found 0 boxes for img
2142/2248
(416, 416, 3)
Found 0 boxes for img
2143/2248
(416, 416, 3)
Found 3 boxes for img
2144/2248
(416, 416, 3)
Found 1 boxes for img
2145/2248
(416, 416, 3)
Found 1 boxes for img
2146/2248
(416, 416, 3)
Found 1 boxes for img
2147/2248
(416, 416, 3)
Found 2 boxes for img
2148/2248
(416, 416, 3)
Found 1 boxes for img
2149/2248
(4

In [27]:
if list_all[1]==None:
    print(1)

1


In [43]:
len(list_all[0])

2

In [46]:
list_f=[]
for i in range(len(list_all)):
        if list_all[i]==None:
            pass
        else:
            for j in range(len(list_all[i])):
                list_f.append(list_all[i][j])

In [49]:
list_f

[['200801213.jpg', 'bucket', 457, 328, 601, 588, 0.67935175],
 ['200801213.jpg', 'bowl', 330, 346, 382, 402, 0.39834145],
 ['200801221.jpg', 'styrofoam', 50, 90, 115, 133, 0.3364409],
 ['20080123.jpg', 'bucket', 270, 193, 540, 480, 0.9838459],
 ['200801241.jpg', 'bucket', 159, 50, 270, 204, 0.87853897],
 ['200801241.jpg', 'bucket', 0, 2, 184, 240, 0.99420995],
 ['200804281.jpg', 'bucket', 183, 0, 213, 27, 0.3579936],
 ['200804303.jpg', 'water_tower', 8, 20, 283, 205, 0.75616205],
 ['2008043043.jpg', 'bucket', 408, 209, 436, 252, 0.6282186],
 ['200805203.jpg', 'plastic_bag', 34, 177, 522, 420, 0.30628103],
 ['200805203.jpg', 'plastic_bag', 3, 137, 151, 190, 0.33408073],
 ['200805203.jpg', 'plastic_bag', 535, 291, 722, 389, 0.35957232],
 ['200805203.jpg', 'plastic_bag', 113, 215, 436, 358, 0.78683925],
 ['200807182.jpg', 'plastic_bag', 234, 170, 290, 234, 0.5892451],
 ['200807182.jpg', 'plastic_bag', 275, 160, 350, 259, 0.65912485],
 ['200807182.jpg', 'plastic_bag', 180, 234, 256, 307, 0

In [50]:
classes = ["aquarium","bottle","bowl","box","bucket","plastic_bag","plate","styrofoam","tire","toilet","tub","washing_machine","water_tower"]
len(classes)
for i in range(len(list_f)):
    for j in range(len(classes)):
        if list_f[i][1]==classes[j]:
            list_f[i][1]=j+1
            break

In [51]:
df = pd.DataFrame(list_f, columns = ["image_filename", "label_id","x","y","w","h","confidence"])

In [52]:
df

,image_filename,label_id,x,y,w,h,confidence
0,200801213.jpg,5,457,328,601,588,0.679352
1,200801213.jpg,3,330,346,382,402,0.398341
2,200801221.jpg,8,50,90,115,133,0.336441
3,20080123.jpg,5,270,193,540,480,0.983846
4,200801241.jpg,5,159,50,270,204,0.878539
...,...,...,...,...,...,...,...
2920,2018081464.jpg,3,39,6,361,225,0.348212
2921,2018081464.jpg,3,99,52,297,201,0.707617
2922,2018081468.jpg,9,83,116,270,246,0.534445
2923,2018081468.jpg,9,172,93,399,225,0.852425


In [53]:
df.to_csv('test.csv',index=0)